In [1]:
import os
import glob
import pyodbc
import pandas as pd

# Database connection parameters
server = r'Som-PC\SQLSERVER2022'
database = 'Python-SQL Project'

# Use Windows Authentication
conn_str = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes"
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Folder containing CSV files
folder_path = r"D:/Python SQL Data Analyst"

# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Process each CSV file
for file in csv_files:
    table_name = os.path.splitext(os.path.basename(file))[0]  # Use filename as table name
    print(f"\U0001F4C2 Processing file: {file} -> Table: {table_name}")

    # Read CSV into DataFrame
    df = pd.read_csv(file)

    # Replace NaN and empty strings with None (SQL NULL)
    df = df.where(pd.notna(df), None)
    df.replace("", None, inplace=True)

    # Ensure numeric columns contain valid values and limit float precision
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        df[col] = df[col].astype(str).str.replace(',', '', regex=True)  # Remove commas
        df[col] = pd.to_numeric(df[col], errors='coerce').round(5)  # Convert invalid numbers to NaN and limit precision
    
    # Create table dynamically based on DataFrame columns
    column_types = []
    for col in df.columns:
        dtype = df[col].dtype
        if dtype == 'int64':
            sql_type = 'INT'
        elif dtype == 'float64':
            sql_type = 'DECIMAL(18,5)'  # More precise than FLOAT
        elif dtype == 'bool':
            sql_type = 'BIT'
        elif dtype == 'datetime64':
            sql_type = 'DATETIME'
        else:
            sql_type = 'NVARCHAR(MAX)'  # Ensures no truncation of string data
        column_types.append(f"[{col}] {sql_type}")

    columns_sql = ", ".join(column_types)
    create_table_sql = f"""
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
    BEGIN
        CREATE TABLE {table_name} ({columns_sql});
    END
    """

    try:
        cursor.execute(create_table_sql)
        conn.commit()
        print(f"\U00002705 Table '{table_name}' created successfully.")
    except Exception as e:
        print(f"⚠️ Error creating table: {e}")

    # Insert Data into Table
    inserted_rows = 0
    for index, row in df.iterrows():
        columns = ', '.join(f"[{col}]" for col in df.columns)
        placeholders = ', '.join(['?' for _ in df.columns])
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        
        try:
            cursor.execute(sql, tuple(row))
            inserted_rows += 1
        except pyodbc.IntegrityError:
            print(f"⚠️ Skipping duplicate row {index} in '{table_name}'.")
        except pyodbc.ProgrammingError as pe:
            print(f"⚠️ Data type mismatch at row {index} in '{table_name}': {pe}")
        except Exception as e:
            print(f"⚠️ Error inserting row {index} in '{table_name}': {e}")
    
    conn.commit()
    print(f"\U00002705 {inserted_rows} rows inserted into '{table_name}' successfully.")

# Close connection
cursor.close()
conn.close()
print("🎉 All files processed successfully!")

📂 Processing file: D:/Python SQL Data Analyst\customers.csv -> Table: customers
✅ Table 'customers' created successfully.
✅ 99441 rows inserted into 'customers' successfully.
📂 Processing file: D:/Python SQL Data Analyst\geolocation.csv -> Table: geolocation
✅ Table 'geolocation' created successfully.
✅ 1000163 rows inserted into 'geolocation' successfully.
📂 Processing file: D:/Python SQL Data Analyst\orders.csv -> Table: orders
✅ Table 'orders' created successfully.
✅ 99441 rows inserted into 'orders' successfully.
📂 Processing file: D:/Python SQL Data Analyst\order_items.csv -> Table: order_items
✅ Table 'order_items' created successfully.
✅ 112650 rows inserted into 'order_items' successfully.
📂 Processing file: D:/Python SQL Data Analyst\payments.csv -> Table: payments
✅ Table 'payments' created successfully.
✅ 103886 rows inserted into 'payments' successfully.
📂 Processing file: D:/Python SQL Data Analyst\products.csv -> Table: products
✅ Table 'products' created successfully.
⚠️